In [1]:
import os
import json
import sys
from typing import List
from pydantic import BaseModel, ValidationError
import multiprocessing as mp # 병렬 처리를 위해 추가
import time
# Google Drive 마운트 및 경로 설정 (사용자 환경에 맞게 조정 필요)
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    # common 폴더가 있는 경로 추가 (layers.py 등이 있는 곳)
    sys.path.append('/content/drive/MyDrive/DL_project/common')
    # ocr_dataset.py가 있는 경로 추가
    sys.path.append('/content/drive/MyDrive/DL_project/dataset')
    BASE_GDRIVE_PATH = "/content/drive/MyDrive/DL_project"
except ImportError:
    print("Google Colab 환경이 아닙니다. 경로를 직접 설정해주세요.")
    BASE_GDRIVE_PATH = "YOUR_PROJECT_PATH" # 예: "/path/to/your/DL_project"
    sys.path.append(BASE_GDRIVE_PATH)
    sys.path.append(os.path.join(BASE_GDRIVE_PATH, "dataset"))


# --- ocr_dataset.py에서 필요한 Pydantic 모델 정의 가져오기 ---
# (ocr_dataset.py 파일에 있는 정의와 동일해야 합니다)
class Annotation(BaseModel):
    object_recognition: int
    text_language: int

class Dataset(BaseModel):
    category: int
    identifier: str
    label_path: str
    name: str
    src_path: str
    type: int

class Images(BaseModel):
    acquistion_location: str
    application_field: str
    background: int
    data_captured: str
    height: int
    identifier: str
    media_type: int
    pen_color: str
    pen_type: int
    type: str
    width: int
    writer_age: int
    writer_sex: int
    written_content: int

class BBox(BaseModel):
    data: str
    id: int
    x: List[int]
    y: List[int]

class Label(BaseModel):
    Annotation: Annotation
    Dataset: Dataset
    Images: Images
    bbox: List[BBox]
# -----------------------------------------------------------

# --- 실제 최대 길이 찾는 함수 ---
def find_max_len_in_file(label_path):
    """단일 JSON 파일에서 가장 긴 bbox.data 길이를 찾습니다."""
    max_len = 0
    try:
        with open(label_path, 'r', encoding='utf-8') as f:
            label_data = json.load(f)
            label = Label(**label_data)
            for bbox in label.bbox:
                current_len = len(bbox.data)
                if current_len > max_len:
                    max_len = current_len
    except (FileNotFoundError, ValidationError, json.JSONDecodeError) as e:
        # print(f"파일 처리 오류 {label_path}: {e}") # 오류 발생 시 주석 해제
        pass # 오류가 있는 파일은 건너뜁니다.
    return max_len

def find_overall_max_length(label_dir):
    """주어진 디렉토리의 모든 JSON 라벨 파일에서 최대 길이를 찾습니다."""
    if not os.path.isdir(label_dir):
        print(f"오류: '{label_dir}'는 유효한 디렉토리가 아닙니다.")
        return 0

    label_files = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith('.json')]

    if not label_files:
        print(f"오류: '{label_dir}' 디렉토리에서 JSON 파일을 찾을 수 없습니다.")
        return 0

    print(f"총 {len(label_files)}개의 라벨 파일 분석 시작...")

    # 멀티프로세싱을 사용하여 병렬로 처리
    num_processes = mp.cpu_count()
    print(f"{num_processes}개의 프로세스를 사용하여 병렬 처리합니다.")

    with mp.Pool(processes=num_processes) as pool:
        max_lengths = pool.map(find_max_len_in_file, label_files)

    overall_max_len = 0
    if max_lengths:
        overall_max_len = max(max_lengths)

    return overall_max_len

# --- 실행 부분 ---
if __name__ == "__main__":
    # ✨✨✨ 훈련 라벨 디렉토리 경로를 설정하세요 ✨✨✨
    # TRAIN_LBL_PATH = os.path.join(BASE_GDRIVE_PATH, "train_labels") # Google Drive 경로
    TRAIN_LBL_PATH = "/content/local_ocr_data/train_labels" # 로컬 경로 예시

    start_time = time.time()
    max_label_length = find_overall_max_length(TRAIN_LBL_PATH)
    end_time = time.time()

    if max_label_length > 0:
        print("\n--- 결과 ---")
        print(f"훈련 데이터셋에서 가장 긴 텍스트 라벨의 길이는: {max_label_length}")
        print(f"분석에 걸린 시간: {end_time - start_time:.2f} 초")

        # 현재 설정된 MAX_LABEL_LEN과 비교 (선택 사항)
        # CURRENT_MAX_LEN = 25 # 현재 코드의 설정값
        # if max_label_length > CURRENT_MAX_LEN:
        #     print(f"⚠️ 경고: 현재 설정된 MAX_LABEL_LEN ({CURRENT_MAX_LEN})보다 긴 라벨이 있습니다. 값을 늘리는 것을 고려하세요.")
        # else:
        #     print(f"✅ 현재 설정된 MAX_LABEL_LEN ({CURRENT_MAX_LEN})은 충분합니다.")

MessageError: Error: credential propagation was unsuccessful